In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

In [3]:
links=[]
para=['1_p','2_p', '3_p', '4_p', '5_p', '6_p','7_p']
url='https://www.zillow.com/beverly-hills-ca-90210/{}/'
for p in para:
  new_link=url.format(p)
  links.append(new_link)
print(links[3])

https://www.zillow.com/beverly-hills-ca-90210/4_p/


In [12]:
price = []
house_info = []
address = []
for url in links:
  response = requests.get(url, headers=header)
  data = response.text
  

  soup = BeautifulSoup(data, 'html.parser')
  amount = soup.find_all('div', class_='list-card-price')
  for i in amount:
      price.append(i.text)

  #response = requests.get(i, headers=header)
  #data = response.text
  #soup = BeautifulSoup(data, 'html.parser')
  beds = soup.find_all('ul',class_='list-card-details')
  for i in beds:
      house_info.append(i.text)

  #response = requests.get(i, headers=header)
  #data = response.text
  #soup = BeautifulSoup(data, 'html.parser')
  x = soup.find_all('address',class_='list-card-addr')
  for i in x:
      address.append(i.text)
house_data = []
house_data.append(price)
house_data.append(house_info)
house_data.append(address)
dict = {'price':house_data[0], 'house_info':house_data[1], 'address':house_data[2]}
data = pd.DataFrame(dict)
print(data.shape)

(160, 3)


In [13]:
df = data.copy()

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   price       160 non-null    object
 1   house_info  160 non-null    object
 2   address     160 non-null    object
dtypes: object(3)
memory usage: 3.9+ KB


In [15]:
df['prices'] = df['price'].apply(lambda x: x[1:])

In [16]:
df[['beds','baths','sq_feet','sqft_unit','type','for','sale','None']] = df['house_info'].str.split(expand=True)

In [17]:
df.head(1)

,price,house_info,address,prices,beds,baths,sq_feet,sqft_unit,type,for,sale,None
0,"$3,189,000","4 bds,5 ba,4,283 sqft- House for sale","3088 Deep Canyon Dr, Beverly Hills, CA 90210","3,189,000",4,"bds,5","ba,4,283",sqft-,House,for,sale,None


In [18]:
df[['redundant','baths']] = df['baths'].str.split(',',n=1,expand=True)

In [19]:
df['sq_feet']=df['sq_feet'].replace('ba,','',regex=True)

In [20]:
df[['street_name','city','state_zip_code']]=df['address'].str.split(',',n=2, expand=True)

In [21]:
df.head(1)

,price,house_info,address,prices,beds,baths,sq_feet,sqft_unit,type,for,sale,None,redundant,street_name,city,state_zip_code
0,"$3,189,000","4 bds,5 ba,4,283 sqft- House for sale","3088 Deep Canyon Dr, Beverly Hills, CA 90210","3,189,000",4,5,"4,283",sqft-,House,for,sale,None,bds,3088 Deep Canyon Dr,Beverly Hills,CA 90210


In [22]:
#drop unnecessary columns
feature_drop = ['price','house_info','for', 'sale','redundant','sqft_unit','None']
df1 = df.drop(feature_drop, axis=1)

In [23]:
df1.head()

,address,prices,beds,baths,sq_feet,type,street_name,city,state_zip_code
0,"3088 Deep Canyon Dr, Beverly Hills, CA 90210","3,189,000",4,5,"4,283",House,3088 Deep Canyon Dr,Beverly Hills,CA 90210
1,"30 Beverly Park Ter, Beverly Hills, CA 90210","110,000,000",8,12,--,House,30 Beverly Park Ter,Beverly Hills,CA 90210
2,"524 N Beverly Dr, Beverly Hills, CA 90210","21,495,000",8,12,"10,750",House,524 N Beverly Dr,Beverly Hills,CA 90210
3,"601 Mountain Dr, Beverly Hills, CA 90210","59,000,000",6,15,"27,470",House,601 Mountain Dr,Beverly Hills,CA 90210
4,"2571 Wallingford Dr, Beverly Hills, CA 90210","85,000,000",12,24,"38,000",House,2571 Wallingford Dr,Beverly Hills,CA 90210


In [24]:
#beakup state_zip_code into state and zip code
#df1['zip_code']=df1['state_zip_code'].apply(lambda x: x[3:])
df1['zip_code'] = df1['state_zip_code'].replace('CA ','', regex=True)

In [25]:
#df1['state']=df1['state_zip_code'].apply(lambda x:x[0:3])

In [26]:
df1.head(1)

,address,prices,beds,baths,sq_feet,type,street_name,city,state_zip_code,zip_code
0,"3088 Deep Canyon Dr, Beverly Hills, CA 90210","3,189,000",4,5,"4,283",House,3088 Deep Canyon Dr,Beverly Hills,CA 90210,90210


In [27]:
#reorder columns
zillow_df = df1[['prices','beds','baths','sq_feet','type','address','street_name',
       'city', 'zip_code']]

# Final Dataset - zillow_df

In [28]:
zillow_df.head()

,prices,beds,baths,sq_feet,type,address,street_name,city,zip_code
0,"3,189,000",4,5,"4,283",House,"3088 Deep Canyon Dr, Beverly Hills, CA 90210",3088 Deep Canyon Dr,Beverly Hills,90210
1,"110,000,000",8,12,--,House,"30 Beverly Park Ter, Beverly Hills, CA 90210",30 Beverly Park Ter,Beverly Hills,90210
2,"21,495,000",8,12,"10,750",House,"524 N Beverly Dr, Beverly Hills, CA 90210",524 N Beverly Dr,Beverly Hills,90210
3,"59,000,000",6,15,"27,470",House,"601 Mountain Dr, Beverly Hills, CA 90210",601 Mountain Dr,Beverly Hills,90210
4,"85,000,000",12,24,"38,000",House,"2571 Wallingford Dr, Beverly Hills, CA 90210",2571 Wallingford Dr,Beverly Hills,90210


In [29]:
zillow_df.shape

(160, 9)